# import

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [4]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [6]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate):
    return tf.keras.layers.Dropout(rate)

# Create model(class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(4):
            self.model.add(dense(512,weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [8]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(4):
        model.add(dense(512,weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [9]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [10]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [11]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.8298, train_loss: 2.39644957, train_accuracy: 0.2031, test_accuracy: 0.1491
Epoch: [ 0] [    1/  468] time: 1.1449, train_loss: 2.37935495, train_accuracy: 0.1953, test_accuracy: 0.1844
Epoch: [ 0] [    2/  468] time: 1.4531, train_loss: 2.22969961, train_accuracy: 0.2500, test_accuracy: 0.2422
Epoch: [ 0] [    3/  468] time: 1.8275, train_loss: 2.27378225, train_accuracy: 0.3672, test_accuracy: 0.2803
Epoch: [ 0] [    4/  468] time: 2.1277, train_loss: 2.28494310, train_accuracy: 0.3438, test_accuracy: 0.3352
Epoch: [ 0] [    5/  468] time: 2.4897, train_loss: 2.29614997, train_accuracy: 0.3672, test_accuracy: 0.3589
Epoch: [ 0] [    6/  468] time: 2.9575, train_loss: 2.22789550, train_accuracy: 0.3906, test_accuracy: 0.4028
Epoch: [ 0] [    7/  468] time: 3.2856, train_loss: 2.21125221, train_accuracy: 0.4766, test_accuracy: 0.4453
Epoch: [ 0] [    8/  468] time: 3.6357, 

Epoch: [ 0] [   74/  468] time: 24.1004, train_loss: 0.65284228, train_accuracy: 0.8594, test_accuracy: 0.8872
Epoch: [ 0] [   75/  468] time: 24.4166, train_loss: 0.54454511, train_accuracy: 0.8672, test_accuracy: 0.8870
Epoch: [ 0] [   76/  468] time: 24.7417, train_loss: 0.66706121, train_accuracy: 0.8906, test_accuracy: 0.8837
Epoch: [ 0] [   77/  468] time: 25.0778, train_loss: 0.54428661, train_accuracy: 0.8906, test_accuracy: 0.8801
Epoch: [ 0] [   78/  468] time: 25.4378, train_loss: 0.64849675, train_accuracy: 0.8516, test_accuracy: 0.8827
Epoch: [ 0] [   79/  468] time: 25.7191, train_loss: 0.43125170, train_accuracy: 0.9297, test_accuracy: 0.8845
Epoch: [ 0] [   80/  468] time: 26.0211, train_loss: 0.59796745, train_accuracy: 0.9141, test_accuracy: 0.8822
Epoch: [ 0] [   81/  468] time: 26.3423, train_loss: 0.56878912, train_accuracy: 0.8906, test_accuracy: 0.8816
Epoch: [ 0] [   82/  468] time: 26.6265, train_loss: 0.64844131, train_accuracy: 0.8750, test_accuracy: 0.8861
E

Epoch: [ 0] [  148/  468] time: 47.0541, train_loss: 0.49315241, train_accuracy: 0.8906, test_accuracy: 0.9196
Epoch: [ 0] [  149/  468] time: 47.3783, train_loss: 0.40007597, train_accuracy: 0.9453, test_accuracy: 0.9228
Epoch: [ 0] [  150/  468] time: 47.8101, train_loss: 0.49380702, train_accuracy: 0.8906, test_accuracy: 0.9238
Epoch: [ 0] [  151/  468] time: 48.2021, train_loss: 0.26708323, train_accuracy: 0.9297, test_accuracy: 0.9255
Epoch: [ 0] [  152/  468] time: 48.6229, train_loss: 0.44528845, train_accuracy: 0.9219, test_accuracy: 0.9257
Epoch: [ 0] [  153/  468] time: 48.9969, train_loss: 0.34877202, train_accuracy: 0.9297, test_accuracy: 0.9260
Epoch: [ 0] [  154/  468] time: 49.4018, train_loss: 0.44828504, train_accuracy: 0.9375, test_accuracy: 0.9260
Epoch: [ 0] [  155/  468] time: 49.8816, train_loss: 0.31163046, train_accuracy: 0.9531, test_accuracy: 0.9274
Epoch: [ 0] [  156/  468] time: 50.3782, train_loss: 0.34314764, train_accuracy: 0.9297, test_accuracy: 0.9286
E

Epoch: [ 0] [  222/  468] time: 71.4627, train_loss: 0.33442625, train_accuracy: 0.9141, test_accuracy: 0.9406
Epoch: [ 0] [  223/  468] time: 71.8167, train_loss: 0.40541220, train_accuracy: 0.9219, test_accuracy: 0.9415
Epoch: [ 0] [  224/  468] time: 72.1857, train_loss: 0.37587059, train_accuracy: 0.8906, test_accuracy: 0.9417
Epoch: [ 0] [  225/  468] time: 72.4829, train_loss: 0.24681994, train_accuracy: 0.9688, test_accuracy: 0.9402
Epoch: [ 0] [  226/  468] time: 72.7861, train_loss: 0.53345209, train_accuracy: 0.9141, test_accuracy: 0.9394
Epoch: [ 0] [  227/  468] time: 73.0885, train_loss: 0.27564448, train_accuracy: 0.9297, test_accuracy: 0.9396
Epoch: [ 0] [  228/  468] time: 73.3838, train_loss: 0.20896764, train_accuracy: 0.9531, test_accuracy: 0.9393
Epoch: [ 0] [  229/  468] time: 73.6840, train_loss: 0.37689808, train_accuracy: 0.9453, test_accuracy: 0.9385
Epoch: [ 0] [  230/  468] time: 73.9722, train_loss: 0.37071913, train_accuracy: 0.9297, test_accuracy: 0.9402
E

Epoch: [ 0] [  296/  468] time: 95.9392, train_loss: 0.17437604, train_accuracy: 0.9609, test_accuracy: 0.9447
Epoch: [ 0] [  297/  468] time: 96.2593, train_loss: 0.23718418, train_accuracy: 0.9531, test_accuracy: 0.9431
Epoch: [ 0] [  298/  468] time: 96.5615, train_loss: 0.32824448, train_accuracy: 0.9219, test_accuracy: 0.9431
Epoch: [ 0] [  299/  468] time: 96.8788, train_loss: 0.24651858, train_accuracy: 0.9766, test_accuracy: 0.9445
Epoch: [ 0] [  300/  468] time: 97.1880, train_loss: 0.29230917, train_accuracy: 0.9766, test_accuracy: 0.9451
Epoch: [ 0] [  301/  468] time: 97.5221, train_loss: 0.27570635, train_accuracy: 0.9609, test_accuracy: 0.9451
Epoch: [ 0] [  302/  468] time: 97.8672, train_loss: 0.36191559, train_accuracy: 0.9375, test_accuracy: 0.9445
Epoch: [ 0] [  303/  468] time: 98.2023, train_loss: 0.20314756, train_accuracy: 0.9453, test_accuracy: 0.9456
Epoch: [ 0] [  304/  468] time: 98.5544, train_loss: 0.39695761, train_accuracy: 0.9219, test_accuracy: 0.9454
E

Epoch: [ 0] [  370/  468] time: 121.2524, train_loss: 0.37231621, train_accuracy: 0.8906, test_accuracy: 0.9482
Epoch: [ 0] [  371/  468] time: 121.5404, train_loss: 0.17616257, train_accuracy: 0.9609, test_accuracy: 0.9496
Epoch: [ 0] [  372/  468] time: 121.8366, train_loss: 0.22500145, train_accuracy: 0.9609, test_accuracy: 0.9523
Epoch: [ 0] [  373/  468] time: 122.1358, train_loss: 0.16858709, train_accuracy: 0.9688, test_accuracy: 0.9552
Epoch: [ 0] [  374/  468] time: 122.4300, train_loss: 0.31213850, train_accuracy: 0.9375, test_accuracy: 0.9560
Epoch: [ 0] [  375/  468] time: 122.7302, train_loss: 0.36274463, train_accuracy: 0.9453, test_accuracy: 0.9555
Epoch: [ 0] [  376/  468] time: 123.0383, train_loss: 0.25307781, train_accuracy: 0.9531, test_accuracy: 0.9547
Epoch: [ 0] [  377/  468] time: 123.3366, train_loss: 0.24049687, train_accuracy: 0.9375, test_accuracy: 0.9554
Epoch: [ 0] [  378/  468] time: 123.6417, train_loss: 0.26838747, train_accuracy: 0.9531, test_accuracy:

Epoch: [ 0] [  444/  468] time: 146.3139, train_loss: 0.31586045, train_accuracy: 0.9766, test_accuracy: 0.9531
Epoch: [ 0] [  445/  468] time: 146.7029, train_loss: 0.24791098, train_accuracy: 0.9219, test_accuracy: 0.9534
Epoch: [ 0] [  446/  468] time: 147.0040, train_loss: 0.34912151, train_accuracy: 0.9375, test_accuracy: 0.9528
Epoch: [ 0] [  447/  468] time: 147.3018, train_loss: 0.38707167, train_accuracy: 0.9297, test_accuracy: 0.9532
Epoch: [ 0] [  448/  468] time: 147.6578, train_loss: 0.21240921, train_accuracy: 0.9531, test_accuracy: 0.9535
Epoch: [ 0] [  449/  468] time: 147.9599, train_loss: 0.19462886, train_accuracy: 0.9609, test_accuracy: 0.9532
Epoch: [ 0] [  450/  468] time: 148.2555, train_loss: 0.19201785, train_accuracy: 0.9609, test_accuracy: 0.9535
Epoch: [ 0] [  451/  468] time: 148.5537, train_loss: 0.19319215, train_accuracy: 0.9766, test_accuracy: 0.9536
Epoch: [ 0] [  452/  468] time: 148.8621, train_loss: 0.29069296, train_accuracy: 0.9375, test_accuracy:

In [12]:
# test acc = 95.14%